In [2]:
from nanover.omni.openmm import OpenMMSimulation
from nanover.openmm import serializer
import numpy as np
import openmm.app as app
import MDAnalysis as mda
from openmm.app import PDBFile

## Creating an initial pdb

In [ ]:
nanotube_xml = "nanotube_3_equilibrated.xml"
nanotube_pdb = "nanotube-methane"
pdb_file_name = nanotube_pdb + ".pdb"

In [ ]:
# Create a pdb from the initial OpenMM simulation
simulation = OpenMMSimulation.from_xml_path(nanotube_xml)
simulation.load()


with open(pdb_file_name, "w") as pdb_file:
    PDBFile.writeFile(simulation.simulation.topology, simulation.simulation.context.getState(getPositions=True).getPositions(), pdb_file)

## Using MDAnalysis to move the molecular system

In [3]:
def center_simulation(u: mda.Universe):
    u.atoms.positions
    u.atoms.center_of_geometry()
    u.atoms.translate(- u.atoms.center_of_geometry())
    u.atoms.center_of_geometry()
    u.atoms.translate([box_length/2, box_length/2, box_length/2])
    u.atoms.center_of_geometry()
    return u

In [21]:
# Create universe from pdb
nanotube_pdb = "nanotube-methane.pdb"
u = mda.Universe(nanotube_pdb)
box_length = 40 # in Angstroms

In [22]:
u = center_simulation(u)

# In VR: x is left(-) to right(+), y is down(-) to up(+), and z is forwards(-) to backwards(+)
u.atoms.translate([0, -2, -5])

<AtomGroup with 65 atoms>

In [23]:
test_pdb_name = nanotube_pdb + "-TEST.pdb"
u.atoms.write(test_pdb_name)

In [24]:
# Create OpenMM simulation
nanotube_xml = "nanotube_3_equilibrated.xml"
simulation = OpenMMSimulation.from_xml_path(nanotube_xml)
simulation.load()

# Update the OpenMM simulation with the new positions
pdb = PDBFile(test_pdb_name)
simulation.simulation.context.setPositions(pdb.positions)

# Write new simulation to xml
with open("nanotube-methane-TEST.xml", 'w') as outfile:
    outfile.write(serializer.serialize_simulation(simulation.simulation))